# Sphinx pro automatické generování dokumentace z docstringů a type hintů
Sphinx pravděpodobně znáte z prohlížení dokumentace různých Python balíčků. V současnosti je automaticky generovaná dokumentace standardem (už nikdo nevyvíjí dokumentaci ručně) a Sphinx je jedním z nejlepších nástrojů pro její generování.

Sphinx je napsaný v Pythonu a je dostupný na PyPI. Instalaci můžete provést pomocí pip:

In [2]:
# !pip install sphinx --break-system-packages

In [1]:
# sphinx má mnoho theme, oblíbené je např.
# !pip install sphinx_rtd_theme --break-system-packages

# Quickstart
Sphinx má pěkný tutoriál, viz [http://www.sphinx-doc.org/en/master/usage/quickstart.html](http://www.sphinx-doc.org/en/master/usage/quickstart.html). 

My si zde ukážeme jen naprosto nejelementárnější použití.

## Složka s projektem
Předpokládáme souborovou strukturu projektu (mujbalik):

```
mujbalik
│
├── mujbalik 
│   ├── knihovna.py
│   ├── literatura.py
│   ├── ucty.py
|   └── __init__.py
└── examples.ipynb
```

# Inicializace Sphinx dokumentace
Vytvořte si v kořenové složce projektu složku `docs`:

```
mujbalik
│
├── mujbalik 
│   ├── knihovna.py
│   ├── literatura.py
│   ├── ucty.py
|   └── __init__.py
├── docs
└── examples.ipynb
```

Přesuňte se do `docs` a v ní spusťte příkaz `sphinx-quickstart`.

Sphinx se vás zeptá na několik věcí, jako je název projektu, jméno autora, verzi, atd.

Po skončení inicializace se vytvoří ve složce `docs` složka `source` a složka `build`. Také se vytvoří makefile, který vám umožní snadno generovat dokumentaci.

## Autodoc pro vytvoření šablony dokumentace dle struktury projektu
Ve složce `source` uvnitř složky `docs` jsou již nyní soubory `.rst`. Mohly bychom již ručně vyrobit strukturu dokumentace. To však nechceme a spokojíme se s automatickou strkturou dokumentace, kterou vytvoříme pomocí Sphinx pluginu `autodoc`.

Ve složce `docs` spusťte příkaz `sphinx-apidoc -o ./source ../mujbalik`. První parametr je cesta k výstupní složce, druhý parametr je cesta k projektu, který chceme dokumentovat (z `docs` je to o jednu složku výše a pak do složky `mujbalik`).

## Generování dokumentace
Pro vygenerování dokumentace je třeba zajistit, že Sphinx umí najít složku s projektem. Toho docílíme pomocí souboru `conf.py`, který se nachází v `docs/source`. V souboru `conf.py` je potřeba přidat aresu na cestu k projektu:

```python
# toto přidáme kamkoliv v souboru
import os
import sys
sys.path.insert(0, os.path.abspath('../..'))
```

Proč přidáváme cestu o dvě složky výše ('../..')? Protože v `docs/source` je soubor `conf.py` a od něj je to o dvě složky výše do rootu projektu.

V souboru `conf.py` můžeme také nastavit theme dokumentace. Výchozí je `alabaster`, ale můžeme si vybrat i jiné. Například `sphinx_rtd_theme`:

```python
html_theme = 'sphinx_rtd_theme'
```

Nyní můžeme dokumentaci vygenerovat pomocí příkazu `make html` v `docs`. Výsledná dokumentace se nachází v `docs/build/html`.